In [1]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial import Polynomial
import os
import pandas as pd
from pathlib import Path
from matplotlib.offsetbox import AnchoredText
from mpl_toolkits import mplot3d

%matplotlib inline

In [2]:
angle = 2.4
Cu_ka = 8047.8227

Mo_ka = (275, 17444.3)
Mo_kb = (309, 19608.3)
W_ka2 = (913, 57981.7)
W_ka1 = (934, 59318.24)
W_kb = (1058, 67244.3)

In [3]:
'''
DESCRIPTION:
Not much of a 'working class', it just helps set up the Sample class a bit easier. A scan is made
up of spectral data with an arbitrary x-axis, which are just bins.
'''

class scan:
    
    '''
    ATTRIBUTES AND INITIALIZING
    '''    
    
    def __init__(self, spectrum, position):
        self.spectrum = spectrum
        self.beam_position = position
    
    '''
    GETTER METHODS
    '''       
    
    def get_spectrum(self): return self.spectrum
    def get_beam_position(self): return self.beam_position
    def get_bins(self): return list(range(len(self.get_spectrum())))
    def get_num_of_points(self): return len(self.get_spectrum())

    '''
    VISUALIZATION
    '''
    
    def quick_plot(self, energy_bool):
        #Plot by bins
        fig, ax = plt.subplots(figsize = (10,6))
        ax.plot(self.get_bins(), self.get_spectrum(), linewidth=2.0)
        ax.set_xlabel('Bins')
        ax.set_ylabel('Counts')
        ax.grid(True)
        return (fig, ax)

In [4]:
class sample:
    
    '''
    ATTRIBUTES AND INITIALIZING
    '''
    
    def __init__(self, sample_name, scan_set):
        #Important all spectra arrays are of the same length. First step is to sanity check that.
        if self._check_dimensions(scan_set) != True: print('Warning: Arrays of different lengths')
        
        #Instantiation attributes and fundamentals
        self.name = sample_name
        self.scan_list = scan_set

        #attributes intrinsic to formulation
        self.recipe_ph = None
        self.recipe_steel = None 
        
        #attributes related to aging
        self.aging_temp = None
        self.aging_atm = None
        self.aging_humidity = None
        self.aging_time = None        
        
        #attributes related to measurements
        self.scans_off_sample = None
        self.sample_edge = None
        
        #attributes for plotting
        self.fit_degree = 3
        self.fit_features = [Mo_ka, Mo_kb, W_ka2, W_ka1, W_kb]
        self.fit_coeff = self._gen_fit_coeff()
        self.units = 'Energy [keV]'
    
    '''
    SETTER METHODS
    '''
    
    #Formulation setters
    def set_ph(self, hi_low_string): self.recipe_ph = hi_low_string
    def set_steel_fiber(self, true_false): self.recipe_steel = true_false
    
    #Aging setters
    def set_temp(self, temperature): self.aging_temp = temperature
    def set_atm(self, atmosphere): self.aging_atm = atmosphere
    def set_humidity(self, rh): self.aging_humidity = rh
    def set_time(self, days): self.aging_time = days
    
    #Formulation + Aging setters catch-all
    def set_recipe_and_exposure(self, ph, steel_fiber, temp, atm, humidity, time):
        self.set_ph(ph) 
        self.set_steel_fiber(steel_fiber)
        self.set_temp(temp)
        self.set_humidity(humidity)
        self.set_time(time)

    #Measurement setters
    def set_scans_off_sample(self, back_list): self.scans_off_sample = back_list
    def set_sample_edge(self, edge_ind): self.sample_edge = edge_ind
    
    #Energy calibration / conversion setters
    def set_fit_degree(self, n): self.fit_degree = n
    def set_fit_features(self, list_of_tup): self.fit_features = list_of_tup
    def set_fit_coeff(self): self.fit_coeff = self._gen_fit_coeff()
    def set_units(self, unit): self.units = units

    '''
    GETTER METHODS
    '''

    #Basic attributes and quick calculation getters
    def get_name(self): return self.name
    def get_scan_list(self): return self.scan_list
    def get_num_of_scans(self): return len(self.get_scan_list())
    def get_bins(self): return self.get_scan_list()[0].get_bins()
    def get_num_of_points(self): return len(self.get_bins())
    def get_scan(self, index): return (self.get_scan_list()[index])
    def get_spectrum(self, index): return (self.get_scan(index).get_spectrum())
    def get_positions(self):
        scan_pos = []
        for a in range(self.get_num_of_scans()):
            if (self.scan_list[a]).beam_position == None: scan_pos.append(0)
            else: scan_pos.append((self.scan_list[a]).beam_position)
        return scan_pos
    
    #Attributes related to chemistry and aging
    def get_ph(self): return (self.recipe_ph)
    def get_steel_fiber(self): return (self.recipe_steel)    
    def get_temp(self): return (self.aging_temp) 
    def get_atm(self): return (self.aging_atm)
    def get_humidity(self): return (self.aging_humidity)
    def get_time(self): return (self.aging_time)
    
    #Measurement / data processing related getters
    def get_units(self): return self.units
    def get_energy(self): return poly_function(self.get_bins(), self.get_fit_coeff())
    def get_scans_off_sample(self): return self.scans_off_sample
    def get_sample_edge(self): return self.sample_edge
    def get_avg_background_spectrum(self): return self._gen_background()
    def get_fit_degree(self): return self.fit_degree
    def get_fit_features(self): return self.fit_features
    def get_fit_coeff(self): self.fit_coeff = self._gen_fit_coeff(); return self.fit_coeff

    #Spit out the important stuff
    def get_attr(self):
        attr_names = ['Name', 'Number of Scans', 'Positions', 'Points per Scan', 'Sample Edge', 'Scans Off Sample',
                      'Conversion Features', 'Degree of Fit', 'Fit Coeffecients', 'pH', 'Steel Fiber', 'Aging Temp.',
                      'Aging Atm.', 'Aging Humidity', 'Aging Time']
        attr_prop = [self.get_name(), self.get_num_of_scans(), self.get_positions(), self.get_num_of_points(),
                     self.get_sample_edge(), self.get_scans_off_sample(), self.get_fit_features(), self.get_fit_degree(),
                     self.get_fit_coeff(), self.get_ph(), self.get_steel_fiber(), self.get_temp(), self.get_atm(), 
                     self.get_humidity(), self.get_time()]
        bigstring = ''
        if len(attr_prop) == len(attr_names):
            for x in range(len(attr_names)):
                bigstring = bigstring + (attr_names[x])
                bigstring = bigstring + (': ')
                bigstring = bigstring + (str(attr_prop[x]))
                bigstring = bigstring + ('\n')
            return (bigstring)
        
    '''
    INTERNAL METHODS
    '''
    
    #Find the specified scan indices that were off the sample and return an average spectrum of them
    def _gen_background(self):
        avg_background_spectra = []
        spectra_points = self.get_num_of_points()
        i_list = self.get_scans_off_sample()
        if i_list is None: return [0]*(spectra_points)
        num_of_scans = len(i_list)
        for x in range(spectra_points):
            avg_background = 0
            for y in range(num_of_scans):
                avg_background = avg_background + self.get_spectrum(i_list[y])[x]
            avg_background_spectra.append(round(float(avg_background / num_of_scans),2))
        return avg_background_spectra
    
    #Returns list of polynomial coeffecients, highest degree first. Needs degree and set of xy points
    def _gen_fit_coeff(self):
        return np.polyfit([i[0] for i in self.get_fit_features()], 
                                      [i[1] for i in self.get_fit_features()], 
                                      self.get_fit_degree())    
    
    #Called during initialization. Make sure every scan in the sample's list is the same length
    def _check_dimensions(self, scan_list):
        same_length = True
        prev_length = scan_list[0].get_num_of_points()
        for c in range(len(scan_list)):
            if c > 1:
                curr_length = scan_list[c].get_num_of_points()
                if curr_length != prev_length:
                    same_length = False
                prev_length = curr_length
        return same_length
    
    def _conv_bins_to_energy(self, bin_array): 
        return poly_function(bin_array, self.get_fit_coeff())
    
    def _convert_units(self, from_unit, to_unit):
        return
    
    #Given a value in kev, find the nearest index / bin closest to the value
    def _find_nearest_index(self, energy_kev):
        en_array = self.get_energy()
        for d in range(len(en_array)):
            if en_array[d] > energy_kev:
                above_val_delta = en_array[d] - energy_kev
                below_val_delta = energy_kev - en_array[d-1]
                if below_val_delta < above_val_delta: return (d-1)
                else: return d
        return print("Error: _find_nearest_index() didn't work")
    
    #given a spectra array, limits in kev, and an interval, return an energy and spectra array that is an
    #intervalled selection of the whole. i.e. in:[10,5,8,32,4,2,7] -> interval 2 -> out:[10,8,32,2]
    def _interval_spectral_slice(self, spec_array, bounds, interval):
        bin_bounds = self._find_nearest_index(bounds[0]) , self._find_nearest_index(bounds[1])
        spec_array_slice = spec_array[bin_bounds[0]:bin_bounds[1]]
        en_data = self._conv_bins_to_energy(list(range(bin_bounds[0],bin_bounds[1])))
        loopcount = bin_bounds[0]
        index = 0
        interval_en = []
        interval_spec = []
        for p in range(bin_bounds[0], bin_bounds[1]):
            if p == loopcount:
                interval_en.append(en_data[index])
                interval_spec.append(spec_array_slice[index])
                loopcount = loopcount + interval
                index = index + interval
        return interval_en, interval_spec
    
#     def _off_sample_background_subtractor(self): return
                                          
#     def _linear_sample_background_subtractor(self): return

#     def _trapezoid_rule_numerical_integral(self, bounds):
#         return
    
#     def _back(self): 
#         return
        
    '''
    DATA PIPELINE: BACKGROUND SELECTION, REMOVAL, AND PEAK INTEGRATION; FOR CLARITY AND VALIDATION
    '''
    
    def peak_integral_pipeline(self, peakbounds, lback, uback, interval):
        
        #STEP 0: Make subplots and broad axis labels, use interval to determine the scans to plot
        fig, axs = plt.subplots(ncols=2, nrows=4, figsize=(9, 14),layout="constrained")
        fig.suptitle(self.get_name())
        domain_offset = 2
        for x in range(3):
            for y in range(2):
                axs[x,y].set_xlim(lback[0] - domain_offset, uback[1] + domain_offset)
                axs[x,y].set_ylim(0, 350)
                axs[x,y].set_xlabel('Energy [keV]')
                axs[x,y].set_ylabel('Counts')

        x_data = self.get_energy()
        start_i = self.get_sample_edge()
        total_scans = self.get_num_of_scans()
        interval_scans = []
        sample_pos_set = []

        for q in range(total_scans):
            if q == start_i:
                interval_scans.append(self.get_spectrum(q))
                sample_pos_set.append(self.get_scan(q).get_beam_position())
                start_i = start_i + interval
        scan_num = len(interval_scans)
        
        
        #STEP 1: Display the spectra, plot only every 1st, 2nd, or 3rd scan etc. given by interval
        for q in range(scan_num):
            axs[0,0].plot(x_data, interval_scans[q], linewidth = 0.5, color = 'b')
        axs[0,0].set_title('Raw Spectra')

        
        #STEP 2: Display the spectra and show the avg off-sample background
        off_sample_background = self.get_avg_background_spectrum()
        axs[0,1].plot(x_data, off_sample_background, linewidth = 0.8, color = 'r')
        for q in range(scan_num):
            axs[0,1].plot(x_data, interval_scans[q], linewidth = 0.5, color = 'b')
        axs[0,1].set_title('Unadjusted Peak Regime and Off-Sample Background Spectrum')
        
        
        #STEP 3: Display the off-sample background subtracted spectra
        adj_spec_set = []
        for q in range(scan_num):
            adj_spec = np.subtract(interval_scans[q], off_sample_background)
            adj_spec_set.append(adj_spec)
            axs[1,0].plot(x_data, adj_spec, linewidth = 0.5, color = 'b')
        axs[1,0].set_title('Off-Sample Background Subtracted')
        
        
        #STEP 4: Highlight selected low and high background ranges for linear background fit
        axs[1,1].axvspan(lback[0], lback[1], alpha=0.5, color='yellow')
        axs[1,1].axvspan(uback[0], uback[1], alpha=0.5, color='yellow')
        scatter_spacing = 5
        low_scat_x, low_scat_y = self._interval_spectral_slice(adj_spec_set[0], lback, scatter_spacing)
        hi_scat_x, hi_scat_y = self._interval_spectral_slice(adj_spec_set[0], uback, scatter_spacing)
        scat_x = low_scat_x + hi_scat_x
        scat_y = low_scat_y + hi_scat_y
        axs[1,1].scatter(scat_x, scat_y, marker = 'x', color = 'r', zorder = 2)
        for q in range(scan_num):
            axs[1,1].plot(x_data, adj_spec_set[q], linewidth = 0.5, color = 'b', zorder = 1)
        axs[1,1].set_title('Further Background Selection')
        
        
        #STEP 5: Fit example line through lower and upper background sections
        coeff_set = []
        degree = 2        
        for f in range(scan_num):
            low_back_xy = self._interval_spectral_slice(adj_spec_set[f], lback, 1)
            hi_back_xy = self._interval_spectral_slice(adj_spec_set[f], uback, 1)
            back_x = low_back_xy[0] + hi_back_xy[0]
            back_y = low_back_xy[1] + hi_back_xy[1]
            coeff_set.append(np.polyfit(back_x, back_y, degree))
        axs[2,0].plot(x_data, np.poly1d(coeff_set[0])(x_data), linewidth = 1, color = 'r', zorder = 2)
        for q in range(scan_num):
            axs[2,0].plot(x_data, adj_spec_set[q], linewidth = 0.5, color = 'b', zorder = 1)
            # axs[2,0].plot(x_data, np.poly1d(coeff_set[q])(x_data), linewidth = 0.5, color = 'r', zorder = 2)
        axs[2,0].set_title('Poly Fit (Degree: ' + str(degree) +') of Remaining Background')
        
        
        #STEP 6: Show final peak after background subtractions
        processed_spectral_set = []
        for q in range(scan_num):
            processed_y = np.subtract(adj_spec_set[q], np.poly1d(coeff_set[q])(x_data))
            processed_spectral_set.append(processed_y)
            axs[2,1].plot(x_data, processed_y, linewidth = 0.5, color = 'b')
        axs[2,1].set_title('Fully Processed Spectral Peak')

            
        #STEP 7: Show final peak bounds and center
        for q in range(scan_num):
            axs[3,0].plot(x_data, processed_spectral_set[q], linewidth = 0.8, color = 'b')
        axs[3,0].set_xlim(peakbounds[0]-2, peakbounds[1]+2)
        axs[3,0].set_ylim(0,350)
        axs[3,0].axvspan(peakbounds[0], peakbounds[1], alpha=0.5, color='green')
        center = (peakbounds[0]+peakbounds[1])/2
        axs[3,0].axvline(x = center, color = 'red', label = 'Peak Bounds Center')
        axs[3,0].set_xlabel('Energy [keV]')
        axs[3,0].set_ylabel('Counts')
        axs[3,0].set_title('Peak Integral Bounds and Center')

        
        #STEP 8: Peak area integral as a function of R
        lower_i = self._find_nearest_index(peakbounds[0])
        upper_i = self._find_nearest_index(peakbounds[1])
        integrate_peak = []
        for k in range(scan_num):
            peak_area = 0.0
            for j in range(lower_i, upper_i):
                avg = (processed_spectral_set[k][j] + processed_spectral_set[k][j+1])/2
                area = avg * (x_data[j+1] - x_data[j])
                peak_area = peak_area + area
            integrate_peak.append(peak_area)
        axs[3,1].plot(sample_pos_set, integrate_peak, linewidth = 1)
        axs[3,1].scatter(sample_pos_set, integrate_peak, marker = '.', color = 'b', zorder = 2)
        axs[3,1].set_xlabel('Distance from Center [mm]')
        axs[3,1].set_ylabel('Peak Area')
        axs[3,1].set_title('Peak Area as a Function of Position')
        
        return fig, axs


In [5]:
'''
INITIALIZATION HELPER FUNCTIONS
'''

def sample_instant_helper(df, scannum):
    scan_index = (df[df['ScanNum']==scannum].index.values)[0]
    pos_array = df['Position'][scan_index]
    spectra_set = df['Spectra'][scan_index]
    sampname = df['FullName'][scan_index]
    pH = df['pH'][scan_index]
    time = int(df['Days'][scan_index])
    temper = int(df['Temperature'][scan_index])
    atmosphere = df['Atmosphere'][scan_index]
    st_bool = df['SteelFiber'][scan_index]
    
    sample_edge = 3
    background_scans = [0,1,2]
    slist = []
    
    for q in range(len(pos_array)):
        radius = pos_array[q]
        spectra = spectra_set[q]
        slist.append(scan(spectra, radius))
        
    samp = sample(sampname, slist)
    
    samp.set_ph(pH)
    samp.set_steel_fiber(st_bool)
    samp.set_temp(temper)
    samp.set_atm(atmosphere)
    samp.set_time(time)
    samp.set_sample_edge(sample_edge)
    samp.set_scans_off_sample(background_scans)
    
    return samp


#When bool_pos_given_as_index is True, the pos instantiation variable is an index. Otherwise, when false,
#the pos instantiation variable is an actual position value, like -13.0 or 6.75. It also catches
#an edge case when the position array is empty, such as the calibration scan 1069. Things may behave funny when
#position arrays end in duplicates of 0s or any duplicates in general
def scan_instant_helper(df, scannum, pos, bool_pos_given_as_index):
    errorflag = True
    pos_i = None
    scan_index = (df[df['ScanNum']==scannum].index.values)[0]
    pos_array = df['Position'][scan_index]
    
    #edge case checker, if the position array of a given scan number is empty
    if ((pos == 0) and len(pos_array)==0):
        spectra_set = df['Spectra'][scan_index]
        spectra = spectra_set
        return scan(spectra, None)

    #If the bool is given as true, treat the pos variable like an index and not a value
    #Check first that the pos as an index is within the range of the scan's position array
    if bool_pos_given_as_index == True:
        if pos > (len(pos_array)-1): 
            return print('ERROR: Index is out of range of position array')
        else:
            pos_i = pos
            pos_value = abs(df['Position'][scan_index][pos_i])
            spectra_set = df['Spectra'][scan_index]
            spectra = spectra_set[pos_i]
            return scan(spectra, pos_value)
    
    #Otherwise, the bool was false and the pos variable was given as an actual position in the array
    #Check first that the pos value is actually in the array, and if so instantiate the scan with appropriate info
    else:
        for a in range(len(pos_array)):
            pos_val = pos_array[a]
            #This old logic in the 'or' statement was to catch the empty edge case, it should be unnecessary now
            if abs(pos_array[a])==abs(pos) or ((pos == 0) and len(pos_array)==0): 
                errorflag = False
                pos_i = a
                
        if errorflag == True: print('ERROR: Position is out of range')
        else:
            spectra_set = df['Spectra'][scan_index]
            spectra = spectra_set[pos_i]
            position_val = abs(df['Position'][scan_index][pos_i])
            return scan(spectra, position_val)
        

In [6]:
'''
OTHER IMPORTANT FUNCTIONS OUTSIDE OF CLASS METHODS
'''

#given x as a single float, int, or list; and a set of polynomial coeffecients, calculate and return the
#poly fit at the given x or list of converted values. Used to convert bin scale to energy

def poly_function(x, coeff):
    order = len(coeff) - 1
    pows = []
    for q in range(order + 1):
        pows.append(order - q)
    if type(x) == int or type(x) == float:
        y = 0
        for r in range(len(pows)):
            y = y + coeff[r] * np.power(x, pows[r])
        return y
    elif type(x) == list and (type(x[0]) == int or type(x[0]) == float):
        y_poly = []
        for s in range(len(x)):
            y = 0
            for t in range(len(pows)):
                y = y + coeff[t] * np.power(x[s], pows[t])
            y_poly.append(y/1000) 
        return y_poly
    else: return print('Error: Expected int, float, or list')

In [7]:
#Make a dataframe from the sample directory. Good to visualize data. You should 
#feed this dataframe to another function that will make an array of objects

def gen_spec_df(folder_dir, meta_dic):
    
    #initialize a df of the CSV sample/scan info.
    #This should probably be something I iron out with path (it'd be a '\' on PC) but it's fine for now
    raw_df = pd.read_csv(folder_dir + "/" + meta_dic)
    
    #get the hdf filename for every file in the directory
    files = []
    for x in os.listdir(folder_dir):
        if x.endswith(".hdf5"):
            files.append(x)
    
    #Add a column for filename in meta_df that is the same length of the meta_df
    edxname = [''] * len(raw_df.index)
    raw_df['FileName'] = edxname
    
    #Find each file in the meta_df by comparing scan number. Add file name 
    for y in range(len(files)):
        hdf_file = h5py.File(directory + files[y], 'r')
        for z in range(len(raw_df.index)):
            if raw_df['ScanNum'].values[z] == hdf_file.attrs['Scan Number']:
                raw_df['FileName'].values[z] = directory+files[y]
    
    #Add a column for EDX data arrays
    edxd = [''] * len(raw_df.index)
    raw_df['Spectra'] = edxd
    raw_df['Position'] = edxd
    
    #Scan through filenames and add position and spectra data arrays to df
    #If there's none, set those values equal to some string or warning
    for h in range(len(raw_df.index)):
        datahdf = h5py.File(raw_df['FileName'].values[h], 'r')
        positionflag = 'No Position Data (uh oh)'
        spectraflag = 'No Spectra Data (uh oh)'
        with h5py.File(raw_df['FileName'].values[h], 'r') as f:
            for x in range(len(f.keys())):
                if list(f.keys())[x] == '7bmb1:aero:m2.VAL':
                    raw_df['Position'].values[h] = f['7bmb1:aero:m2.VAL'][...]
                    positionflag = 'yep'
                if list(f.keys())[x] == 'dxpMercury:mca1.VAL':
                    raw_df['Spectra'].values[h] = f['dxpMercury:mca1.VAL'][...]
                    spectraflag = 'yep'
        if positionflag != 'yep':
            raw_df['Position'].values[h] = []
        if spectraflag != 'yep':
            raw_df['Spectra'].values[h] = []

    return raw_df


In [8]:
'''
Mo W FOIL REFERENCE -- DOES THE ENERGY SCALE MAKE SENSE?
'''

background_scans = [scan_instant_helper(sample_df, 1093, 0, True),
                    scan_instant_helper(sample_df, 1093, 0, True),
                    scan_instant_helper(sample_df, 1093, 0, True)]
mo_w_ref_scan = scan_instant_helper(sample_df, 1069, 0, False)
scan_list = background_scans + [mo_w_ref_scan]
mo_w = sample('Mo W Foil Energy Conversion Reference Scan', scan_list)
mo_w.set_sample_edge(3)
mo_w.set_scans_off_sample([0,1,2])

peak = (82.7, 85)
lower_back = (77, 81)
upper_back = (88, 92)
plot_every_other = 0

unadj_peak_xlim = (0, 100)

x = mo_w.peak_integral_pipeline(peak, lower_back, upper_back, plot_every_other)
x[0].delaxes(x[1][1,0])
x[0].delaxes(x[1][1,1])
x[0].delaxes(x[1][2,0])
x[0].delaxes(x[1][2,1])
x[0].delaxes(x[1][3,0])
x[0].delaxes(x[1][3,1])


x[1][0,0].set_xlim(unadj_peak_xlim[0],unadj_peak_xlim[1])
x[1][0,1].set_xlim(unadj_peak_xlim[0],unadj_peak_xlim[1])
x[1][0,0].set_ylim(0,1300)
x[1][0,1].set_ylim(0,1300)


NameError: name 'sample_df' is not defined

In [9]:
directory = '/Users/anthonygironda/Documents/MDA_Scans_Seidler/'
meta = '7bmb1_10.19.22_EDXindex.csv'

sample_df = gen_spec_df(directory, meta)
display(sample_df)

,ScanNum,FullName,pH,Days,Temperature,Atmosphere,SteelFiber,Aborted,Notes,Start (mm),End (mm),Step Size (mm),Slit Entrance (um),Scan Time,Rotation Speed (deg/s),FileName,Spectra,Position
0,1066,Foil Scan 1,NaN,NaN,NaN,NaN,NaN,False,EDX Commisioning,NaN,NaN,NaN,NaN,NaN,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 221.0, 295.0, 314.0, 334.0, 354.0, 354....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1067,Foil Scan 2,NaN,NaN,NaN,NaN,NaN,False,EDX Commisioning,NaN,NaN,NaN,NaN,NaN,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 203.0, 278.0, 317.0, 330.0, 369.0, 349....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1068,Foil Scan 3,NaN,NaN,NaN,NaN,NaN,False,EDX Commisioning,NaN,NaN,NaN,NaN,NaN,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 849.0, 945.0, 1174.0, 1282.0, 1385.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1069,Mo W Energy Calibration,NaN,NaN,NaN,NaN,NaN,False,EDX Calibration,NaN,NaN,NaN,NaN,90.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 5366.0, 6316.0, 7253.0, 7844.0, 8473.0, ...",[]
4,1070,LaB6,NaN,NaN,NaN,NaN,NaN,False,EDX Commisioning,NaN,NaN,NaN,NaN,90.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 5138.0, 6247.0, 7093.0, 7730.0, 8211.0, ...",[]
5,1071,Calcite (CaCO3),NaN,NaN,NaN,NaN,NaN,False,EDX Calibration,NaN,NaN,NaN,NaN,90.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 5150.0, 6166.0, 7062.0, 7600.0, 8218.0, ...",[]
6,1072,High pH SF 25C 14d,High,14.0,25.0,100% CO2,True,False,Commisioning,13.0,6.0,1.00,250.0,20.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 1054.0, 1230.0, 1444.0, 1533.0, 1597.0, ...",[]
7,1073,High pH SF 25C 14d,High,14.0,25.0,100% CO2,True,False,Commisioning,13.0,6.0,1.00,250.0,20.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 106.0, 120.0, 128.0, 174.0, 164.0, 185....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,1074,High pH SF 25C 14d,High,14.0,25.0,100% CO2,True,False,Commisioning,13.0,6.0,1.00,250.0,20.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 98.0, 110.0, 135.0, 170.0, 159.0, 162.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,1075,High pH SF 25C 14d,High,14.0,25.0,100% CO2,True,False,Commisioning,13.0,6.0,1.00,250.0,20.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 6217.0, 7384.0, 8390.0, 9519.0, 9998.0, ...",[]


In [10]:
#List of good sample nums:
HS602 = sample_instant_helper(sample_df, 1091)
HS604 = sample_instant_helper(sample_df, 1092)
HS606 = sample_instant_helper(sample_df, 1083)
HS6014= sample_instant_helper(sample_df, 1087)

HS250 = sample_instant_helper(sample_df, 1093)
HS254 = sample_instant_helper(sample_df, 1094)
HS256 = sample_instant_helper(sample_df, 1089)
HS259 = sample_instant_helper(sample_df, 1088)

HB602 = sample_instant_helper(sample_df, 1105)
HB604 = sample_instant_helper(sample_df, 1104)
HB606 = sample_instant_helper(sample_df, 1103)

HB250 = sample_instant_helper(sample_df, 1108)
HB254 = sample_instant_helper(sample_df, 1107)
HB2514= sample_instant_helper(sample_df, 1106)

LS602 = sample_instant_helper(sample_df, 1096)
LS604 = sample_instant_helper(sample_df, 1095)

LS250 = sample_instant_helper(sample_df, 1102)
LS252 = sample_instant_helper(sample_df, 1101)
LS254 = sample_instant_helper(sample_df, 1100)
LS256 = sample_instant_helper(sample_df, 1097)
LS259 = sample_instant_helper(sample_df, 1098)
LS2514= sample_instant_helper(sample_df, 1099)

In [11]:
sample_list = [1091, 1092, 1083, 1087, 1093, 1094, 1089, 1088,
               1105, 1104, 1103, 1108, 1107, 1106, 1096, 1095,
               1102, 1101, 1100, 1097, 1098, 1099]

In [20]:
(df, scannum, pos, bool_pos_given_as_index):

SyntaxError: invalid syntax (635807137.py, line 1)

In [16]:
lab6 = scan_instant_helper(sample_df, 1070, 0, True)
calcite = scan_instant_helper(sample_df, 1071, 0, True)

lab6_samp = sample('LaB6 Ref', [lab6])
calc_samp = sample('Calcite Ref', [calcite])

pos_string = 'No Spatial Information'

sample1 = lab6_samp

bins = sample1.get_bins()
energy = sample1.get_energy()

background = [0] * len(bins)#(sample_instant_helper(sample_df, 1108)).get_avg_background_spectrum()


header = '#Bins\tEnergy [keV]\tAvg. Background\t' + pos_string +'\n'

with open(sample1.get_name() +'.txt', 'w') as f:
    f.write(sample1.get_name())
    f.write('\n*********\n')
    f.write(header)
    for x in range(len(energy)):
        f.write(str(bins[x]))
        f.write('\t')
        f.write(str(round(energy[x],3)))
        f.write('\t')
        f.write(str((background[x])))
        f.write('\t')
        f.write(str(round(sample1.get_scan_list()[0].spectrum[x],3)))
        f.write('\t')
        f.write('\n')
    f.close()


In [12]:
def sample_data_file_gen(df, scan_num):
    sample = sample_instant_helper(df, scan_num)
    fname = str(sample.get_name()) + '.txt'
    
    edge = sample.get_sample_edge()
    all_pos = sample.get_positions()
    pos = all_pos[edge:]
    pos_string = ''
    
    attributes = sample.get_attr()
    bins = sample.get_bins()
    energy = sample.get_energy()        
    average_background = sample.get_avg_background_spectrum()
    
    for x in range(len(pos)):
        pos_string += str(pos[x])
        pos_string += '\t'
        
    header = '#Bins\tEnergy [keV]\tAvg. Background\t' + pos_string +'\n'

    with open(fname, 'w') as f:
        f.write(attributes)
        f.write('\n*********\n')
        f.write(header)
        for x in range(len(energy)):
            f.write(str(bins[x]))
            f.write('\t')
            f.write(str(round(energy[x],3)))
            f.write('\t')
            f.write(str((average_background[x])))
            f.write('\t')
            for y in range(len(pos)):
                index = edge + y
                f.write(str(round(sample.get_scan_list()[index].spectrum[x] - average_background[x],3)))
                f.write('\t')
            f.write('\n')
    # return
    return average_background

In [13]:
avg_bck = test_data_file_gen(sample_df, sample_list[x])

NameError: name 'test_data_file_gen' is not defined

In [14]:
for x in range(len(sample_list)):
    sample_data_file_gen(sample_df, sample_list[x])

In [ ]:
example = sample_instant_helper(sample_df, 1070)
example.set_scans_off_sample([0,1,2])
example.set_sample_edge(3)
avg_off_sample_back = example.get_avg_background_spectrum()

# print(avg_off_sample_back)

peak = (82, 86)
lower_back = (77, 81)
upper_back = (88, 92)

plot_every_other = 30
unadj_peak_xlim = (50, 110)

# example.get_attr()

x = example.peak_integral_pipeline(peak, lower_back, upper_back, plot_every_other)
x[1][0,0].set_xlim(unadj_peak_xlim[0],unadj_peak_xlim[1])
x[1][0,1].set_xlim(unadj_peak_xlim[0],unadj_peak_xlim[1])

for x in range ((example.get_num_of_scans())):
    
    pass
